In [ ]:
from Core import Workspace
import Logger
from Vw import Vw, VwInput
from VwSweep import VwSweep
import VwOptsGrid

logger = Logger.console_logger(0, 'INFO')
context = Workspace(r'C:\data\xbox\xbetdealshomespotlight\ws', logger)

vw_path = 'vw'

vw = Vw(vw_path, context)

sweep = VwSweep(vw)


In [ ]:
input_d = r'C:\data\xbox\xbetdealshomespotlight\train-5-11-2020_5-12-2020\sample-0.004.allslots.tsv.json'

base = {'#base': '--ccb_explore_adf --epsilon 0 --dsjson -b 20'}

In [ ]:
opts_out = ['--cache_file']
input_c = vw.test(input_d, base, opts_out).Populated[0]['--cache_file']
logger.info('Done')

In [ ]:
base = {'#base': '--ccb_explore_adf --epsilon 0 --dsjson --power_t 0 --cb_type ips --coin', "#int": '-q bc'}

In [ ]:
from VwOptsGrid import dimension, product, Grid, Configuration
import pandas as pd

hyper_points = product(
    [base],
    dimension('-l', [1e-5, 1e-4]),
    dimension('--l1', [1e-09, 1e-07])
)

In [ ]:
results = sweep.iteration(hyper_points, input_c)
results = pd.DataFrame(map(lambda r: {'Loss': r.Loss, 'Args': r.Opts}, results)).sort_values('Loss')

In [ ]:
results['Args'].values